In [1]:
cd ./American_dad_script/


C:\Users\berte\Desktop\M2_AIC\NLP\Projet\American_dad_script


In [2]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
# load ascii text and covert to lowercase

full_text = []

for filename in os.listdir(os.getcwd()):
    if (filename!='sample_data' and filename!='.config'):
        raw_text = open(filename, 'r', encoding='utf-8').read()
        full_text.append(raw_text.lower())
flat_list = (', '.join(full_text))
print(len(flat_list))
flat_list = flat_list[:100000].rstrip()

Using TensorFlow backend.


4194580


In [3]:
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ.?!')
answer = ''.join(filter(whitelist.__contains__, flat_list))


In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(answer)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
print(char_to_int)

{' ': 0, '!': 1, '.': 2, '?': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29}


In [6]:
# summarize the loaded data
n_chars = len(answer)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = answer[i:i + seq_length]
	seq_out = answer[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="../weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model

Total Characters:  96368
Total Vocab:  30
Total Patterns:  96268


In [16]:
model.load_weights('../WEIGHT_NLP/weights-improvement-40-1.2952-bigger.hdf5')

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

for i in range(10): 
    # pick a random seed
    #start = numpy.random.randint(0, len(dataX)-1)
    pattern = dataX[i*100 -3]
    print ("Seed:")
    print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    # generate characters
    for i in range(100):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    print ("\nDone.")

Seed:
" ve been up for days making potato saiad.  i cant think straight! my back is kiiiing me!  i know how  "
it was she starte. i want to me? well i know i know ie this is any the coris.  what are 